In [1]:
## MUHAMMAD FARIHEEN BIN ABD RAHIM (SW01082818)
## MUHAMMAD ADEEB BIN ABDULLAH (SW01082814)

In [21]:
## Question 1

In [13]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords

# Download NLTK Stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Define Cleaning Function
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'[^a-z\s]', '', text)  # Remove non-alphabetic chars
    tokens = text.split()
    filtered = [word for word in tokens if word not in stop_words]
    return ' '.join(filtered)

# Load and Sample Dataset
df = pd.read_csv("Reviews.csv")
df = df.sample(n=5000, random_state=42).reset_index(drop=True)

# Clean the Review Text
df['Processed_Text'] = df['Text'].apply(clean_text)

# Save cleaned data
df.to_csv("cleaned_reviews.csv", index=False)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\60139\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#question 2

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF Vectorization
vectorizer = TfidfVectorizer(max_features=500)
X = vectorizer.fit_transform(df['Processed_Text'])

# Convert to DataFrame
tfidf_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

# Combine Only Essential Columns (Score + TF-IDF)
final_df = pd.concat([df[['Score']].reset_index(drop=True), tfidf_df], axis=1)

# Save 
final_df.to_csv("reviews_tfidf.csv", index=False)

In [5]:
pip install textblob

   ---------------------------------------- 0.0/624.3 kB ? eta -:--:--
   ---------------- ----------------------- 262.1/624.3 kB ? eta -:--:--
   ---------------------------------------- 624.3/624.3 kB 2.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mdade\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
pip install vaderSentiment

Note: you may need to restart the kernel to use updated packages.


In [ ]:
#question 3

In [1]:
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.metrics import classification_report
from tabulate import tabulate
import pandas as pd

# Load dataset
df = pd.read_csv("cleaned_reviews.csv")

# Use only 'Processed_Text' and 'Score'
data = list(zip(df["Processed_Text"], df["Score"]))

# Map Score to Actual Sentiment Labels (1-2: negative, 3: neutral, 4-5: positive)
def score_to_label(score):
    if score <= 2:
        return "negative"
    elif score == 3:
        return "neutral"
    else:
        return "positive"

# Initialize table
table_data = [["Text", "Actual Label", "TextBlob Polarity", "TextBlob Sentiment",
               "VADER Compound", "VADER Sentiment"]]

# Initialize VADER analyzer
analyzer = SentimentIntensityAnalyzer()

# Process each row
for text, score in data:
    actual_label = score_to_label(score)

    # TextBlob
    blob = TextBlob(text)
    tb_polarity = blob.sentiment.polarity
    if tb_polarity > 0:
        tb_label = 'positive'
    elif tb_polarity < 0:
        tb_label = 'negative'
    else:
        tb_label = 'neutral'

    # VADER
    vs = analyzer.polarity_scores(text)
    vader_compound = vs['compound']
    if vader_compound > 0.05:
        vader_label = 'positive'
    elif vader_compound < -0.05:
        vader_label = 'negative'
    else:
        vader_label = 'neutral'

    # Append row to table
    table_data.append([text, actual_label, tb_polarity, tb_label, vader_compound, vader_label])

# Print tabular data
print(tabulate(table_data[:30], headers="firstrow", tablefmt="plain"))  # Limit to 30 rows for preview

# Classification report for TextBlob
tb_actual = [row[1] for row in table_data[1:]]
tb_predicted = [row[3] for row in table_data[1:]]
tb_classification_report = classification_report(tb_actual, tb_predicted, target_names=['negative', 'neutral', 'positive'])

# Classification report for VADER
vader_actual = [row[1] for row in table_data[1:]]
vader_predicted = [row[5] for row in table_data[1:]]
vader_classification_report = classification_report(vader_actual, vader_predicted, target_names=['negative', 'neutral', 'positive'])

# Print classification reports
print("\nClassification Report for TextBlob:")
print(tb_classification_report)

print("\nClassification Report for VADER:")
print(vader_classification_report)


Text                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [3]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# Load the reviews_tfidf.csv dataset
df = pd.read_csv("reviews_tfidf.csv")

# Map numeric 'Score' to sentiment categories
def map_score_to_sentiment(score):
    if score <= 2:  # Negative
        return 'negative'
    elif score == 3:  # Neutral
        return 'neutral'
    else:  # Positive
        return 'positive'

df['Sentiment'] = df['Score'].apply(map_score_to_sentiment)

# Separate the features (TF-IDF values) and labels (sentiment scores)
X = df.drop(['Score', 'Sentiment'], axis=1)  # Features (TF-IDF values)
y = df['Sentiment']  # Labels (Mapped sentiment)

# Split data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Initialize classifiers
nb_classifier = MultinomialNB()
svm_classifier = SVC(kernel='linear')

# Train classifiers
nb_classifier.fit(X_train, y_train)
svm_classifier.fit(X_train, y_train)

# Predict sentiment on test data
nb_predictions = nb_classifier.predict(X_test)
svm_predictions = svm_classifier.predict(X_test)

# Calculate and display classification report for Naive Bayes
nb_classification_report = classification_report(y_test, nb_predictions, target_names=['negative', 'neutral', 'positive'])

# Calculate and display classification report for SVM
svm_classification_report = classification_report(y_test, svm_predictions, target_names=['negative', 'neutral', 'positive'])

# Print classification report for Naive Bayes
print("\nClassification Report for Naive Bayes:")
print(nb_classification_report)

# Print classification report for SVM
print("\nClassification Report for SVM:")
print(svm_classification_report)



Classification Report for Naive Bayes:
              precision    recall  f1-score   support

    negative       1.00      0.01      0.01       310
     neutral       0.00      0.00      0.00       165
    positive       0.76      1.00      0.87      1525

    accuracy                           0.76      2000
   macro avg       0.59      0.34      0.29      2000
weighted avg       0.74      0.76      0.66      2000


Classification Report for SVM:
              precision    recall  f1-score   support

    negative       0.60      0.28      0.39       310
     neutral       1.00      0.01      0.02       165
    positive       0.80      0.98      0.88      1525

    accuracy                           0.79      2000
   macro avg       0.80      0.42      0.43      2000
weighted avg       0.79      0.79      0.73      2000



C:\Users\mdade\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\mdade\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\mdade\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [5]:
#Muhammad Fariheen Bin Abd Rahim (SW01082818)
#Muhammad Adeeb Bin Abdullah (SW01082814)

# Model Evaluation Comments (question 4)

# 1. Evaluate the performance of each model using the following metrics:
#    - Precision
#    - Recall
#    - F1-Score
#    - Accuracy

# Lexicon-Based Method (TextBlob)
# Accuracy = 0.79
# Precision: High for 'positive' class (0.83), low for 'neutral' class (0.10), and moderate for 'negative' class (0.56).
# Recall: High for 'positive' class (0.94), but low for 'neutral' (0.03) and 'negative' (0.40).
# F1-Score: Best for 'positive' (0.88), moderate for 'negative' (0.46), and low for 'neutral' (0.04).
# The model performs well on 'positive' sentiments but struggles with 'negative' and 'neutral' ones.

# Lexicon-Based Method (VADER)
# Accuracy = 0.78
# Precision: Moderate for 'positive' class (0.82), but poor for 'neutral' (0.12) and 'negative' (0.58).
# Recall: Very high for 'positive' class (0.95), but poor for 'neutral' (0.04) and 'negative' (0.29).
# F1-Score: High for 'positive' (0.88), moderate for 'negative' (0.39), and low for 'neutral' (0.06).
# Similar to TextBlob, VADER performs best for the 'positive' sentiment but fails to capture 'neutral' and 'negative' effectively.

# Machine Learning (Naive Bayes)
# Accuracy = 0.76
# Precision: Very high for 'negative' (1.00) but very low for 'neutral' (0.00) and moderate for 'positive' (0.76).
# Recall: Perfect for 'positive' (1.00), but very poor for 'negative' (0.01) and 'neutral' (0.00).
# F1-Score: Very low for 'negative' (0.01), best for 'positive' (0.87), and poor for 'neutral' (0.00).
# Naive Bayes performs extremely well in detecting 'positive' reviews but fails to classify 'negative' and 'neutral' correctly.

# Machine Learning (SVM)
# Accuracy = 0.79
# Precision: Good for 'positive' (0.80), but poor for 'neutral' (1.00) and 'negative' (0.60).
# Recall: Excellent for 'positive' (0.98), but very poor for 'neutral' (0.01) and 'negative' (0.28).
# F1-Score: High for 'positive' (0.88), moderate for 'negative' (0.39), and very low for 'neutral' (0.02).
# SVM performs well for the 'positive' sentiment and struggles with 'negative' and 'neutral' classes.

# 2. Comparison of the models:
# - TextBlob and VADER: Both lexicon-based models perform well for 'positive' sentiment classification but fail for 'neutral' and 'negative' sentiments.
# - Naive Bayes: Performs well on 'positive' sentiment classification, with an outstanding recall (1.00) for 'positive' but completely fails at detecting 'neutral' and 'negative' sentiments.
# - SVM: Similar to Naive Bayes, SVM performs well for 'positive' sentiment classification and has a good F1-score for the 'positive' class, but struggles with 'negative' and 'neutral' sentiments.

# 3. Conclusion:
# - Based on accuracy, TextBlob and SVM are the best models (0.79 each).
# - F1-Score for positive sentiment is the highest for both TextBlob (0.88) and VADER (0.88), making them more effective for sentiment analysis in general, especially for detecting positive reviews.
# - Naive Bayes gives perfect results for positive sentiment, but its performance on negative and neutral sentiments makes it unreliable.
# - SVM also shows high performance in positive sentiment classification, though it needs improvement in other categories.


In [7]:
# 5. Discussion: Strengths and Weaknesses of the Selected Models for Sentiment Classification
#Muhammad Fariheen Bin Abd Rahim (SW01082818)
#Muhammad Adeeb Bin Abdullah (SW01082814)

# Lexicon-Based Methods:
# - TextBlob:
#   # Strengths:
#     - Simple and easy to implement, requiring minimal pre-processing.
#     - Works well for applications where quick sentiment analysis is needed.
#     - Performs best for classifying positive sentiments.
#   # Weaknesses:
#     - Struggles with detecting 'neutral' and 'negative' sentiments, as seen in the low recall and F1-score for these classes.
#     - Not suitable for highly nuanced or domain-specific text where word meanings might differ.
#     - Performs poorly on imbalanced datasets, as it fails to properly classify less frequent classes like 'neutral' and 'negative'.
#     - Relatively less accurate compared to machine learning-based approaches in terms of overall precision and recall.
  
# - VADER:
#   # Strengths:
#     - Designed specifically for sentiment analysis, making it more reliable for short texts, like social media posts or reviews.
#     - Performs well on 'positive' sentiment, with very high recall and precision for this class.
#     - Robust to slang, emoticons, and emojis, making it more adaptable to informal language.
#   # Weaknesses:
#     - Like TextBlob, VADER has difficulty classifying 'neutral' and 'negative' sentiments effectively.
#     - Suffers from the same issues with class imbalance, leading to low precision and recall for certain classes.
#     - It may fail to understand the context or sarcasm in certain texts, limiting its accuracy in more complex sentiment analysis tasks.
  
# Machine Learning Methods:
# - Naive Bayes:
#   # Strengths:
#     - Very fast to train and test, making it suitable for real-time applications.
#     - Performs well for 'positive' sentiment classification, with a very high recall.
#     - Simple model, often performing better on well-structured, clean datasets.
#   # Weaknesses:
#     - Poor performance for 'negative' and 'neutral' classes, as seen in the very low precision and recall.
#     - Assumes feature independence, which doesn't hold true for natural language, leading to suboptimal results in some cases.
#     - Not well-suited for more complex texts or datasets with a lot of variability or nuance.
#     - Highly sensitive to imbalanced datasets, where it can overfit the majority class and ignore minority classes.
  
# - SVM (Support Vector Machine):
#   # Strengths:
#     - High accuracy and precision for detecting 'positive' sentiments, especially in imbalanced datasets.
#     - Effective in high-dimensional spaces, making it ideal for text classification tasks with a large number of features (e.g., bag-of-words or TF-IDF).
#     - Can be fine-tuned with kernel functions and regularization parameters to improve performance.
#   # Weaknesses:
#     - Performance is poor for 'neutral' and 'negative' classes, with very low recall.
#     - SVMs are computationally expensive and can be slow to train on very large datasets.
#     - Sensitive to the choice of kernel and hyperparameters, which can make the model difficult to tune.
#     - Prone to overfitting if the dataset is noisy or not properly pre-processed, requiring careful parameter optimization.

